In [1]:
# Run in terminal or command prompt
# python3 -m spacy download en

import numpy as np
import pandas as pd
import re, nltk, spacy, gensim

# Sklearn
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import matplotlib.pyplot as plt
%matplotlib inline

#Gensim
from gensim.models import CoherenceModel

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

import os
root = os.path.expanduser('~')

company_index='5'
company_name='Amazon'
img_path = root + '/Desktop/workspace/indeed/Job-Satisfaction/img/companies/'+f'{company_index}_{company_name}'

if not os.path.exists(img_path):
    os.makedirs(img_path)

In [2]:
## making dataset

def get_px_data(company_index=5, company_name='Amazon', uni=False, bi=False, tri=False):
    data_path = root + '/Desktop/workspace/indeed/Job-Satisfaction/data/companies/'+f'{company_index}_{company_name}'+'/output_data/px_data/pros.csv'
    df = pd.read_csv(data_path)
    print(df.head())
    if uni and bi and tri:
        df_data = df['Unigrams'].fillna('') + ' ' + df['Bigrams'].fillna('') + ' ' + df['Trigrams'].fillna('')
    elif uni:
        df_data = df['Unigrams'].dropna()
    elif bi:
        df_data = df['Bigrams'].dropna()
    elif tri:
        df_data = df['Trigrams'].dropna()
    
    return df_data.tolist()
    
def merge_data(company_list=[5], uni=False, bi=False, tri=False):
    sentences = []
    df_company_list = pd.read_csv( root + '/Desktop/workspace/indeed/Job-Satisfaction/data/scraper_data/review_site.csv')
    sentences = []
    for company_index in company_list:
        company_name = df_company_list.iloc[company_index - 1]['Company_Name']
        sentences.extend(get_px_data(company_index=company_index, company_name=company_name, 
                        uni=uni, bi=bi, tri=tri))
    return sentences

sentences = merge_data(company_list=[5, 6, 7, 8, 9, 10],
                        uni=True, bi=True, tri=True)
# sentences = merge_data(company_list=range(1, 51),
#                         uni=True, bi=True, tri=True)
print(len(sentences))
sentences[:5]

   Unnamed: 0                                        Review_Text  \
0           0                                3 Days in a row off   
1           1  Benefits, There many different shifts, work ha...   
2           2                   Free drinks, paid time, overtime   
3           3  On your own, flexible, can keep job even worki...   
4           4                                     Great benefits   

                                            Unigrams               Bigrams  \
0                                                day                   NaN   
1  advanc benefit differ half hard holiday major ...                   NaN   
2                       drink free overtim paid time  free_drink paid_time   
3                                 flexibl month time          flexibl_time   
4                                            benefit                   NaN   

  Trigrams  
0      NaN  
1      NaN  
2      NaN  
3      NaN  
4      NaN  
   Unnamed: 0                               

['day  ',
 'advanc benefit differ half hard holiday major shift time  ',
 'drink free overtim paid time free_drink paid_time ',
 'flexibl month time flexibl_time ',
 'benefit  ']

In [3]:
data_px = sentences

In [4]:
vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             # max_features=50000,             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_px)
print(data_vectorized)

  (0, 148)	1
  (1, 557)	1
  (1, 491)	1
  (1, 322)	1
  (1, 263)	1
  (1, 257)	1
  (1, 254)	1
  (1, 158)	1
  (1, 41)	1
  (1, 11)	1
  (2, 379)	2
  (2, 376)	1
  (2, 229)	2
  (2, 170)	2
  (2, 557)	2
  (3, 346)	1
  (3, 223)	2
  (3, 557)	2
  (4, 41)	1
  (5, 469)	1
  (6, 355)	2
  (6, 56)	2
  (8, 423)	1
  (8, 258)	2
  (8, 17)	1
  :	:
  (53235, 385)	1
  (53236, 41)	1
  (53237, 564)	1
  (53237, 523)	1
  (53237, 41)	1
  (53238, 583)	1
  (53238, 469)	1
  (53239, 319)	2
  (53239, 229)	2
  (53240, 544)	2
  (53240, 561)	1
  (53240, 41)	2
  (53241, 333)	1
  (53242, 482)	1
  (53242, 466)	1
  (53242, 324)	1
  (53242, 266)	1
  (53242, 477)	1
  (53242, 223)	1
  (53243, 527)	1
  (53243, 544)	1
  (53243, 324)	1
  (53244, 324)	1
  (53245, 392)	2
  (53245, 41)	2


In [5]:
# Materialize the sparse data
data_dense = data_vectorized.todense()

# Compute Sparsicity = Percentage of Non-Zero cells
print("Sparsicity: ", ((data_dense == 0).sum()/data_dense.size)*100, "%")

Sparsicity:  99.64511973307748 %


In [7]:
# Build LDA Model
lda_model = LatentDirichletAllocation(n_topics=8,               # Number of topics
                                      max_iter=100,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=42,          # Random state
                                      batch_size=128,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=100, mean_change_tol=0.001,
             n_components=10, n_jobs=-1, n_topics=8, perp_tol=0.1,
             random_state=42, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)


In [8]:
# Log Likelyhood: Higher the better
print("Log Likelihood: ", lda_model.score(data_vectorized))

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", lda_model.perplexity(data_vectorized))

# Compute Coherence Score
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_vectorized, dictionary=id2word, coherence='u_mass')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

# See model parameters
pprint(lda_model.get_params())

Log Likelihood:  -786081.6060611812
Perplexity:  113.4880234995769
{'batch_size': 128,
 'doc_topic_prior': None,
 'evaluate_every': -1,
 'learning_decay': 0.7,
 'learning_method': 'online',
 'learning_offset': 10.0,
 'max_doc_update_iter': 100,
 'max_iter': 100,
 'mean_change_tol': 0.001,
 'n_components': 10,
 'n_jobs': -1,
 'n_topics': 8,
 'perp_tol': 0.1,
 'random_state': 42,
 'topic_word_prior': None,
 'total_samples': 1000000.0,
 'verbose': 0}


In [13]:
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda_model, data_vectorized, vectorizer, sort_topics=False)
panel

/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.091261  0.100452       1        1   9.867908
1     -0.068384  0.110799       2        1   8.077084
2      0.284233 -0.034241       3        1  11.683671
3     -0.066786  0.113805       4        1   8.045385
4     -0.087065  0.119471       5        1  10.334655
5     -0.167290 -0.424183       6        1  24.821237
6      0.310311 -0.058721       7        1  14.376308
7     -0.113759  0.072619       8        1  12.793753, topic_info=    Category          Freq        Term         Total  loglift  logprob
41   Default  14819.000000     benefit  14819.000000  30.0000  30.0000
385  Default  11450.000000         pay  11450.000000  29.0000  29.0000
229  Default   6503.000000        free   6503.000000  28.0000  28.0000
319  Default   5260.000000       lunch   5260.000000  27.0000  27.0000
557  Default   4034.000000        time   4034.000000  26.0000  26.0000
392  Default   3575.000000       peopl   3575.000000  25.0000  25.0000
223  Default   3275.000000     flexibl   3275.000000  24.0000  24.0000
266  Default   3481.000000        hour   3481.000000  23.0000  23.0000
162  Default   3081.000000    discount   3081.000000  22.0000  22.0000
379  Default   2513.000000        paid   2513.000000  21.0000  21.0000
56   Default   2264.000000       break   2264.000000  20.0000  20.0000
477  Default   2356.000000     schedul   2356.000000  19.0000  19.0000
355  Default   1718.000000        nice   1718.000000  18.0000  18.0000
366  Default   1696.000000    opportun   1696.000000  17.0000  17.0000
148  Default   1974.000000         day   1974.000000  16.0000  16.0000
192  Default   2050.000000     environ   2050.000000  15.0000  15.0000
582  Default   1524.000000       vacat   1524.000000  14.0000  14.0000
183  Default   1640.000000     employe   1640.000000  13.0000  13.0000
487  Default   1497.000000      servic   1497.000000  12.0000  12.0000
11   Default   1314.000000      advanc   1314.000000  11.0000  11.0000
264  Default   1692.000000        home   1692.000000  10.0000  10.0000
544  Default   1298.000000        team   1298.000000   9.0000   9.0000
400  Default   1178.000000       phone   1178.000000   8.0000   8.0000
144  Default   1299.000000      custom   1299.000000   7.0000   7.0000
237  Default   1225.000000         fun   1225.000000   6.0000   6.0000
298  Default   1329.000000       learn   1329.000000   5.0000   5.0000
324  Default   1851.000000       manag   1851.000000   4.0000   4.0000
138  Default   1734.000000      cowork   1734.000000   3.0000   3.0000
227  Default    889.000000        food    889.000000   2.0000   2.0000
470  Default   1005.000000      salari   1005.000000   1.0000   1.0000
..       ...           ...         ...           ...      ...      ...
229   Topic8   6502.115843        free   6503.024977   2.0561  -1.1844
319   Topic8   5259.193601       lunch   5260.102659   2.0560  -1.3966
148   Topic8   1974.002091         day   1974.911251   2.0558  -2.3765
603   Topic8    729.235846        week    730.144899   2.0550  -3.3723
102   Topic8    728.832986       coffe    729.741884   2.0550  -3.3729
524   Topic8    676.566354       stock    677.475535   2.0549  -3.4473
72    Topic8    632.393271        care    633.302499   2.0548  -3.5148
368   Topic8    433.541878      option    434.451041   2.0541  -3.8923
597   Topic8    331.955785        wage    332.865065   2.0535  -4.1593
520   Topic8    330.238844       start    331.147972   2.0535  -4.1645
63    Topic8    329.025244        busi    329.934565   2.0535  -4.1682
604   Topic8    285.657473     weekend    286.566588   2.0530  -4.3095
445   Topic8    280.220191        rate    281.129322   2.0530  -4.3287
34    Topic8    280.350842      awesom    281.260400   2.0530  -4.3283
564   Topic8    184.886820      travel    185.796207   2.0513  -4.7446
359   Topic8    180.797369    occasion    181.706440   2.0512  -4.7669
208   Top

In [14]:
pyLDAvis.save_html(panel, 'a.html')